In [26]:
from collections import defaultdict
from functools import cache
import heapq as heap
from tqdm import tqdm

with open("input.txt", "r") as f:
    lines = f.read()

walls = set()
start = (0, 0)
end = (0, 0)

track = set()

for y, line in enumerate(lines.split("\n")):
    for x, char in enumerate(line):
        if char == "#":
            walls.add((x, y))
        elif char == "S":
            start = (x, y)
            track.add((x, y))
        elif char == "E":
            end = (x, y)
            track.add((x, y))
        else:
            track.add((x, y))


directions = [(0, -1), (1, 0), (0, 1), (-1, 0)]


def get_cost(track, walls, start, end, cheat=None):

    costs = {(node[0], node[1]): float("inf") for node in track} | {
        (wall[0], wall[1]): float("inf") for wall in walls
    }
    costs[(start[0], start[1])] = 0

    visited = set()
    pq = []
    heap.heappush(pq, (0, (start[0], start[1])))

    while pq:

        _, node = heap.heappop(pq)
        visited.add(node)

        for direction in directions:

            cost = 1

            next_node = (node[0] + direction[0], node[1] + direction[1])

            if cheat and node == cheat[0]:
                next_node = cheat[1]
                manhattan_distance = abs(next_node[0] - node[0]) + abs(
                    next_node[1] - node[1]
                )
                cost = manhattan_distance 
            else:
                if next_node not in track:
                    continue

            if next_node in visited:
                continue

            newCost = costs[node] + cost

            if costs[next_node] > newCost:
                costs[next_node] = newCost
                heap.heappush(pq, (newCost, next_node))

    min_dir = costs
    return min_dir


original_costs = get_cost(track, walls, start, end)

max_dim = max([x[1] for x in walls])
cheats = []

original_cost = original_costs[(end[0], end[1])]

def get_savings_over_100(max_steps):

    tot = 0

    for cheat_start in track:
        for dx in range(-max_steps, max_steps + 1):
            for dy in range(-max_steps, max_steps + 1):
                manhattan_distance = abs(dx) + abs(dy)
                if manhattan_distance <= max_steps and (cheat_end := (cheat_start[0] + dx, cheat_start[1] + dy)) in track:
                    cheat_start_cost = original_costs[cheat_start]
                    cheat_end_cost = original_costs[cheat_end]

                    potential_savings = (original_cost - cheat_end_cost) + manhattan_distance + cheat_start_cost

                    if (
                        cheat_start_cost + manhattan_distance < cheat_end_cost
                        and manhattan_distance > 1
                        and potential_savings <= (original_cost - 100)
                    ):
                        tot += 1

    return tot


print(get_savings_over_100(2))
print(get_savings_over_100(20))

1323
983905
